In [1]:
import hashlib
import itertools

from Crypto.Cipher import AES
from tqdm import tqdm

In [2]:
def xor(var, key):
    key, var = key[: len(var)], var[: len(key)]
    int_var = int.from_bytes(var, "little")
    int_key = int.from_bytes(key, "little")
    int_enc = int_var ^ int_key
    return int_enc.to_bytes(len(var), "little")


# Generate `arr` of 20 hashes
arr = [hashlib.sha256(b"PALINDROME IS THE BEST!").digest()]
for i in range(1, 20):
    arr.append(hashlib.sha256(arr[i - 1]).digest())

In [3]:
# Function to generate key based on a given list of indices of hashes in `arr` to use
def gen_key(counters, key=b"\x00" * 32):
    for i in counters:
        key = xor(key, arr[i])
    return key


# Generate all possible keys
keys = []
for i in range(21):
    for x in itertools.combinations(list(range(0, 20)), i):
        keys.append(gen_key(x))

print(len(keys))

1048576


In [4]:
# Function to check if a key is valid by attempted to decrypt ciphertext with given key
def check_key(key):
    cipher = AES.new(key, AES.MODE_GCM, b"PALINDROME ROCKS")  # nonce
    cipher.update(b"welcome_message")
    try:
        dec = cipher.decrypt_and_verify(
            b"\xad\xac\x81\x20\xc6\xd5\xb1\xb8\x3a\x2a\xa8\x54\xe6\x5f\x9a\xad\xa4\x39\x05\xd9\x21\xae\xab\x50\x98\xbd\xe4\xc8\xe8\x2a\x3c\x63\x82\xe3\x8e\x5d\x79\xf0\xc6\xf4\xf2\xe7",
            b"\xbd\xfc\xc0\xdb\xd9\x09\xed\x66\x37\x34\x75\x11\x75\xa2\x7a\xaf",
        )  # ciphertext, tag
        print(dec.decode())
        return dec
    except ValueError:
        return False


# Check all keys
for key in tqdm(keys):
    if check_key(key):
        break

 27%|██▋       | 284167/1048576 [00:35<01:35, 8033.25it/s]

TISC{K3ysP4ce_1s_t00_smol_d2g7d97agsd8yhr}
